<a href="https://colab.research.google.com/github/tanviredu/CompleteDataAnalystCourse/blob/main/LangchainCourse_PART1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
from google.colab import output
!apt-get update && apt-get install -y curl
!curl -fsSL https://ollama.com/install.sh | sh
!ollama serve > /dev/null 2>&1 &
!curl http://localhost:11434/api/version
!ollama pull llama2
from openai import OpenAI
MODEL = "llama2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
response = openai.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "2+2 = ?"}]
)

print(response.choices[0].message.content)
import time
time.sleep(3)
output.clear()

In [2]:
!wget https://raw.githubusercontent.com/tanviredu/Mastering-LangChain/refs/heads/main/0-Data/paul_graham_essay3.txt
!wget https://raw.githubusercontent.com/tanviredu/Mastering-LangChain/refs/heads/main/0-Data/paul_graham_short.txt

--2025-02-11 06:03:33--  https://raw.githubusercontent.com/tanviredu/Mastering-LangChain/refs/heads/main/0-Data/paul_graham_essay3.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘paul_graham_essay3.txt’

paul_graham_essay3. 100%[===================>]  73.28K  --.-KB/s    in 0.02s   

2025-02-11 06:03:33 (3.52 MB/s) - ‘paul_graham_essay3.txt’ saved [75042/75042]

--2025-02-11 06:03:33--  https://raw.githubusercontent.com/tanviredu/Mastering-LangChain/refs/heads/main/0-Data/paul_graham_short.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTT

In [3]:
!pip install langchain
!pip install langchain-community
!pip install faiss-cpu
!pip install langchain-openai

print("Installed Successfully")
time.sleep(4)
output.clear()

In [4]:
from langchain.text_splitter        import CharacterTextSplitter
from langchain.document_loaders     import TextLoader
from langchain_openai               import OpenAIEmbeddings,ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores         import FAISS
from langchain.chains               import RetrievalQA
from langchain.schema import Document

In [5]:
file_path = "./paul_graham_short.txt"
loader = TextLoader(file_path)
documents = loader.load()
print(documents[0])
time.sleep(8)
output.clear()

In [7]:
text_splitter = CharacterTextSplitter(chunk_size=10, chunk_overlap=3)
splits_docs = text_splitter.split_documents(documents)

In [8]:
embedding = OllamaEmbeddings(model="llama2")

<ipython-input-8-9de4dbba1323>:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="llama2")


In [9]:
vectore_store =FAISS.from_documents(splits_docs,embedding)

In [11]:
from langchain_community.llms import Ollama
# Use FAISS as a retriever
retriever = vectore_store.as_retriever()

# Set up a Retrieval-based QA system using LLaMA
qa_chain = RetrievalQA.from_chain_type(
    llm=Ollama(model="llama2"),  # Change model to "llama3" or "mistral" if needed
    retriever=retriever,
    return_source_documents=True
)

<ipython-input-11-b1ad7ddb42dc>:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2"),  # Change model to "llama3" or "mistral" if needed


In [12]:
query = input("What you want to know")
result = qa_chain.invoke({"query":query})
print("Answer: ")
print(result['result'])

What you want to knowwhere tanvir study?
Answer: 
{'query': 'where tanvir study?', 'result': 'Sure, I can answer your question! Based on the context you provided, Tanvir Rahman studied at Chittagong University of Engineering and Technology (CUET).', 'source_documents': [Document(id='b079659a-c6a0-40c2-acc2-c687fbd6702c', metadata={'source': './paul_graham_short.txt'}, page_content='My Name is Tanvir Rahman. My age is 30. I studied in Chittagong University of Engineering and Technology. In short form it is called CUET. I live in Chittagong.')]}


KeyboardInterrupt: Interrupted by user



```
❓ Does RetrievalQA Modify the LLaMA Model with New Data?
No, it does not update the LLaMA model itself. Instead, it retrieves relevant documents from FAISS and uses them as context for generating answers.

🔍 What Happens When You Use RetrievalQA?
1️⃣ LLaMA stays the same – It is not fine-tuned or retrained.
2️⃣ New data is stored in FAISS, not in the model.
3️⃣ At query time, the retriever fetches relevant chunks from FAISS and feeds them to LLaMA.
4️⃣ LLaMA generates answers based on retrieved documents instead of relying only on its pre-trained knowledge.

📝 Example: How It Works
Imagine you have a document saying:

"Elon Musk founded SpaceX in 2002."

If you ask GPT-based LLaMA (without FAISS):
❌ "When was SpaceX founded?"
It might hallucinate (make up an answer) if it lacks this knowledge.

But if you store the document in FAISS and ask the same question using RetrievalQA:
✅ FAISS retrieves the text chunk, and LLaMA uses it to generate a more accurate answer.

🔹 Key Takeaway
LLaMA model remains unchanged – it is not retrained or updated.
New knowledge is added to FAISS, not LLaMA.
RetrievalQA enables real-time document-based Q&A without modifying the model.
🚀 Think of it like an external "memory" for LLaMA!
```

